### Génération d’embeddings

##### 1. Charger les données 

In [2]:
import pandas as pd

data_train = pd.read_csv('../data/processed/train_text_cleaned.csv')
data_test = pd.read_csv('../data/processed/test_text_cleaned.csv')

##### 2. Le choix de modèles génération d’embeddings

Parmi les meilleurs modèles de génération d’embeddings, on trouve :

| Modèle              | Qualité         | Multilingue       | Vitesse |
| ------------------- | --------------- | ----------------- | ------- |
| BGE-large-m3        | Excellent       | Oui               | Lent |
| E5-large-v2         | Excellent       | EN only           | Moyen |
| Multilingual-E5     | Bien            | Oui               | Moyen |
| GTE-large           | Bien            | EN only           | Rapide |
| MiniLM-L12-v2       | qualité faible  | Oui               | Très rapide |


On va utiliser E5-large-v2 pour trouver un équilibre entre la performance et la vitesse d’entraînement.

##### 3. Charger le modèle `e5-large-v2` avec **Sentence Transformers**

Le modèle E5 demande de préfixer les textes avec “query:” ou “passage:” selon l’usage.
- Pour encoder un document → "passage: votre texte ici"
- Pour encoder une requête → "query: votre texte ici"

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-large-v2")

data_train["prefixed_text"] = "passage: " + data_train["text_final"]
data_test["prefixed_text"] = "passage: " + data_test["text_final"]

c:\Users\anass\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anass\.cache\huggingface\hub\models--intfloat--e5-large-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
